# 2. Compute Infection Rates

This notebook contains the workflow for the second milestone in the Manning liveProject *Build Network Models for Pandemics.* *Build Network Models for Pandemics* is part of the series *Federated Learning Over Networks for Pandemics*.

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from geopy.distance import geodesic

## Read data and create network (copied from milestone 1)

In [2]:
csvpath = "../../data/" # path to csv file

In [3]:
df = pd.read_csv(csvpath + "PeopleLocations_p2.csv", delimiter=";") # import data

In [4]:
G = nx.Graph()

for idx, person in df.iterrows():
    # format the coordinates
    coord = np.array((person["Lat"],person["Lon"]))
    
    # convert and combine the date and time
    date = datetime.strptime(person["Date"], "%d-%m-%Y").date()
    time = datetime.strptime(person["Time"], "%H:%M:%S").time()
    date_time = datetime.combine(date,time)
    
    # convert y and n into integers
    if person["Covid19"]=="y":
        rate = 1
    else:
        rate = 0
    
    # add node and create attributes
    G.add_node(idx)
    G.nodes[idx]["name"] = person["ID"]
    G.nodes[idx]["coords"] = coord
    G.nodes[idx]["timestamp"] = date_time
    G.nodes[idx]["Rate"] = rate

In [5]:
for i in G.nodes:
    for j in G.nodes:
        if G.nodes[i] != G.nodes[j]:
            distance = geodesic(G.nodes[i]["coords"], G.nodes[j]["coords"]).m # .m gets the distance from coordinates in meters
            if distance < 2:
                G.add_edge(i,j)

## Doing the updates for 10 iterations

In [6]:
niter = 10

for n in range(niter):
    # I need to copy current rates because I will keep overwriting in these loops
    current_rates = np.fromiter(nx.get_node_attributes(G,'Rate').values(), dtype=float)
    for i in G.nodes:
        neighbors = list(G.neighbors(i))
        W_ii = 1 # the W[i,i] before anything subtracted
        W_ij_term = 0 # what will become sum(W[i,j]*Rate[j]) by the end of the inner loop
        for j in neighbors:
            # note that I am skipping non-neighbors altogether because W[i,j] * Rate[j] = 0 anyway
            W_ii -= 1/(max(G.degree(i), G.degree(j))+1) # W[i,i] - 1/(max(d(i),d(j))+1). The sum will accumulate as loop progresses
            W_ij_term += 1/(max(G.degree(i), G.degree(j))+1) * current_rates[j] # W[i,j] * Rate[j]
        G.nodes[i]["Rate"] = W_ii * current_rates[i] + W_ij_term # Eq1

## Printing the updated "Rates"

In [7]:
for i in G.nodes:
    print("Rates for",G.nodes[i]["name"],":",np.round(G.nodes[i]["Rate"],4))

Rates for Person1 : 0.3012
Rates for Person2 : 0.3011
Rates for Person3 : 0.3013
Rates for Person4 : 0.3012
Rates for Person5 : 0.301
Rates for Person6 : 0.3016
Rates for Person7 : 0.301
Rates for Person8 : 0.3011
Rates for Person9 : 0.1994
Rates for Person10 : 0.3011
Rates for Person11 : 0.3011
Rates for Person12 : 0.3011
Rates for Person13 : 0.3013
Rates for Person14 : 0.3011
Rates for Person15 : 0.3013
Rates for Person16 : 0.3011
Rates for Person17 : 0.3011
Rates for Person18 : 0.3013
Rates for Person19 : 0.301
Rates for Person20 : 0.3019
Rates for Person21 : 0.3012
Rates for Person22 : 0.3014
Rates for Person23 : 0.3014
Rates for Person24 : 0.3011
Rates for Person25 : 0.301
Rates for Person26 : 0.3013
Rates for Person27 : 0.3011
Rates for Person28 : 0.3053
Rates for Person29 : 0.3011
Rates for Person30 : 0.3011
Rates for Person31 : 0.3011
Rates for Person32 : 0.3015
Rates for Person33 : 0.3013
Rates for Person34 : 0.3016
Rates for Person35 : 0.3011
Rates for Person36 : 0.3013
Rates